# Store locations
*__Finding the optimal location for a low-cost supermarket in Madrid__*

## Table of Contents

* [Introduction](#introduction)
    * [Background](#background)
    * [Business problem](#business-problem)
* [Data](#data)
    * [Wikipedia](#wikipedia)
    * [Geospatial data](#geospatial-data)
    * [Census data](#census-data)
* [Methodology](#methodology)
    * [Data acquisition](#data-acquisition)
        * [Scrapping Wikipedia](#scrapping-wikipedia)
        * [Requesting Foursquare API for geospatial data](#requesting-foursquare)
        * [Loading census data file](#loading-census)
    * [Neighbourhood segmentation](#neighbourhood-segmentation)
    * [Census analysis](#census-analysis)
* [Results and discussion](#results-and-discussion)
* [Conclusion](#conclusion)

## Introduction <a class="anchor" id="introduction"></a>

### Background <a class="anchor" id="background"></a>

Opening a physical store is not without risks. One the most obvious risks to evaluate before opening a new store is receiving enough customers to make the business profitable. Therefore it is essential to pick the right location to make sure it is convenient for the customers and there is enough demand for a new store in the area.

Identifying the target customers is also key taking into account the behaviour of the different groups of consumers and the stores they usually buy at.

### Business problem <a class="anchor" id="business-problem"></a>

In this project, we tackle the problem of a low-cost supermarket chain trying to decide in which area of Madrid (Spain) they should open their new store in order to maximise the revenue. It is important to note that the city of Madrid consists of 21 districts and 131 neighbourhoods with great differences between them. 

The goal is to identify the optimal neighbourhood for opening a store taking different factors into consideration such as the types of neighbourhood (a residential area would be ideal), the amount of people living in those areas (the higher the population the higher the food demand), their average income (working class people are preferred) and the stores that are already available (avoiding areas with a big density of supermarkets).

Using data science, geospatial analysis and machine learning techniques, this project aims to provide a solution for this problem and recommending the best neighbourhood for opening the low-cost supermarket.

## Data <a class="anchor" id="data"></a>

The following sections describe the data that is needed for answering this business question.

### Wikipedia <a class="anchor" id="wikipedia"></a>

The first data that we need is the list of neighbourhoods in Madrid. Even though this information could have been directly obtained from a CSV file from Madrid city council, it has been decided to use web scraping on Wikipedia for learning purposes.

The Wikipedia page “List of neighbourhoods of Madrid” shows a table with the name of each neighbourhood for each of the 21 districts. In our project, we will work directly with the neighbourhoods and ignore the districts since this way we can perform a more granular analysis of the areas.

<p align="center">
  <img width="500" height="500" src="img/wikipedia_madrid.png">
</p>

<p style="text-align: center;">
    Source: 
    <a href="https://en.wikipedia.org/wiki/List_of_neighborhoods_of_Madrid">https://en.wikipedia.org/wiki/List_of_neighborhoods_of_Madrid</a>
</p>

### Geospatial data <a class="anchor" id="geospatial-data"></a>

Since the plan is to target residential areas, we need to analyse the type of food venues present in each neighbourhood. With the Foursquare API we can explore the different food venues, considering that a big density of bars and restaurants over very few supermarkets will most likely refer to a business or recreational area where people don’t usually buy at supermarkets. In the other hand, a large proportion of supermarkets over the rest of food venues might indicate it is a residential area where people normally make their food shopping.

Before we can make use of the Foursquare API we need to convert the neighbourhood names into a pair of latitude and longitude coordinates. We can acquire this with the _geocoder_ Python library.
We can query the Foursquare API using the HTTP GET method on the explore endpoint indicating the geographical coordinates, venue categories and radius.

<p align="center">
  <img width="500" height="500" src="img/foursquare_api.png">
</p>

<p style="text-align: center;">
    Source: 
    <a href="https://developer.foursquare.com/docs/api-reference/venues/explore/">https://developer.foursquare.com/docs/api-reference/venues/explore/</a>
</p>

### Census data <a class="anchor" id="census-data"></a>

Finally we will need data from the census of Madrid. We can obtained this data from Excel files that are accessible from the Madrid city council website. Particularly we are interested in the population and the average income of each neighbourhood.

<p align="center">
  <img width="500" height="500" src="img/census_population.png">
</p>

<p style="text-align: center;">
    Source: 
    <a href="http://www-2.munimadrid.es/TSE6/control/seleccionDatosBarrio">http://www-2.munimadrid.es/TSE6/control/seleccionDatosBarrio</a>
</p>

<p align="center">
  <img width="500" height="500" src="img/census_income.png">
</p>

<p style="text-align: center;">
    Source: 
    <a href="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=d029ed1e80d38610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default">https://datos.madrid.es/portal/site/egob</a>
</p>

## Methodology <a class="anchor" id="methodology"></a>

The following sections describe the different stages of this project, starting with the acquisition of the data (venues and census), following with the clustering of the neighbourhoods and finishing with an analysis of the census for the corresponding group of neighbourhoods.

### Libraries import

In [207]:
# for HTTP requests
import requests  

# for HTML scrapping 
from bs4 import BeautifulSoup 

# for table analysis
import pandas as pd
import numpy as np

# for finding geographical coordinates
!pip install geocoder
import geocoder

# for converting an address into latitude and longitude values
from geopy.geocoders import Nominatim

# for rending maps
import folium

# for removing Spanish accents from map labels in Folium (encoding incompatibility)
import unicodedata

### Data acquisition <a class="anchor" id="data-acquisition"></a>

The following sub-sections describe the methodology to obtain the different datasets, either by loading an external file or by scrapping a website.

#### Scrapping Wikipedia <a class="anchor" id="scrapping-wikipedia"></a>

__URL for Wikipedia article__

In [208]:
# URL of wikipedia page from which to scrap tabular data.
wiki_url = "https://en.wikipedia.org/wiki/List_of_neighborhoods_of_Madrid"

__Request & Response__

In [209]:
# If the request was successful, reponse should be '200'.
response = requests.get(wiki_url) #.json()
response

<Response [200]>

__Wrangling HTML With BeautifulSoup__

In [210]:
# Parse response content to html
soup = BeautifulSoup(response.content, 'html.parser')
#soup

__Viewing HTML content__

In [211]:
# Title of Wikipedia page
title = soup.title.string
print(f'Page title: {title}') 

# Find the right table to scrap
wiki_table=soup.find('table', {"class":'wikitable sortable'})

# Get the 1st row of the table i.e. the header
row0 = wiki_table.findAll("tr")[0]

# Show the column names (we're only interested in "Name" i.e. the 4th column)
header = [th.text.rstrip() for th in row0.find_all('th')]
print(f'Column names: {header}') 

Page title: List of neighborhoods of Madrid - Wikipedia
Column names: ['District name (number)', 'District location', 'Number', 'Name', 'Image']


__Scraping the table contents__

In [258]:
# Placeholder for list of neighbourhoods
madrid_neighbourhoods = []

# Iterate through the rows of the table
# Note: each district has sub-rows of neighbourhoods
for row in wiki_table.findAll("tr"):    
    cells = row.findAll('td')

    # Parse 1st neighbourhood of the district
    if len(cells)==5:
        madrid_neighbourhoods.append(cells[3].find(text=True).replace('\n', '').rstrip().lstrip())
    
    # Parse sub-rows (rest of neighbourhoods of the district)
    elif len(cells)==3:
        madrid_neighbourhoods.append(cells[1].find(text=True).replace('\n', '').rstrip().lstrip())
    
print(f'Number of neighbourhoods: {len(madrid_neighbourhoods)}')
madrid_neighbourhoods

Number of neighbourhoods: 131


['Palacio',
 'Embajadores',
 'Cortes',
 'Justicia',
 'Universidad',
 'Sol',
 'Imperial',
 'Acacias',
 'Chopera',
 'Legazpi',
 'Delicias',
 'Palos de Moguer',
 'Atocha',
 'Pacífico',
 'Adelfas',
 'Estrella',
 'Ibiza',
 'Jerónimos',
 'Niño Jesús',
 'Recoletos',
 'Goya',
 'Fuente del Berro',
 'La Guindalera',
 'Lista',
 'Castellana',
 'El Viso',
 'Prosperidad',
 'Ciudad Jardín',
 'Hispanoamérica',
 'Nueva España',
 'Castilla',
 'Bellas Vistas',
 'Cuatro Caminos',
 'Castillejos',
 'Almenara',
 'Valdeacederas',
 'Berruguete',
 'Gaztambide',
 'Arapiles',
 'Trafalgar',
 'Almagro',
 'Ríos Rosas',
 'Vallehermoso',
 'El Pardo',
 'Fuentelarreina',
 'Peñagrande',
 'Pilar',
 'La Paz',
 'Valverde',
 'Mirasierra',
 'El Goloso',
 'Casa de Campo',
 'Argüelles',
 'Ciudad Universitaria',
 'Valdezarza',
 'Valdemarín',
 'El Plantío',
 'Aravaca',
 'Los Cármenes',
 'Puerta del Ángel',
 'Lucero',
 'Aluche',
 'Campamento',
 'Cuatro Vientos',
 'Las Águilas',
 'Comillas',
 'Opañel',
 'San Isidro',
 'Vista Alegre

__Creating a dataframe__

In [259]:
df_madrid = pd.DataFrame(madrid_neighbourhoods, columns=['Neighbourhood'])
print(f'Shape: {df_madrid.shape}')
df_madrid.head()

Shape: (131, 1)


,Neighbourhood
0,Palacio
1,Embajadores
2,Cortes
3,Justicia
4,Universidad


In [260]:
##################################################################################################################################
# DUMMY
radius = 2000
df_madrid = df_madrid.sample(frac=0.1)
##################################################################################################################################

__Getting geographical coordinates for the neighbourhoods__

In [261]:
# Function that retrieves the geographical coordinates for a given neighborhood
def get_coordinates(row):
    # initialize variable to None
    lat_lng_coords = None

    # loop until we get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis(f'{row["Neighbourhood"]}, Madrid')
      lat_lng_coords = g.latlng
    
    # return pair lat,long
    return pd.Series([lat_lng_coords[0], lat_lng_coords[1]])

In [262]:
# Fill coordinates for each row
df_madrid[['Latitude','Longitude']] = df_madrid.apply(get_coordinates, axis=1)
df_madrid.head()

,Neighbourhood,Latitude,Longitude
67,San Isidro,40.39588,-3.73048
68,Vista Alegre,40.38492,-3.74635
122,Rosas,40.42869,-3.60086
38,Arapiles,40.43286,-3.70840
40,Almagro,40.43296,-3.69153


In [263]:
# Make sure we found the coordinates for all the neighbourhoods
df_madrid.isnull().sum()

Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

__Visualizing the neighbourhoods of Madrid__

In [264]:
address = 'Madrid, Spain'
geolocator = Nominatim(user_agent="madrid_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinates of Madrid are {latitude}, {longitude}')

The geograpical coordinates of Madrid are 40.4167047, -3.7035825


In [265]:
# Spanish accents are not correctly rendered in the map labels so we decided
# to remove them for a better understanding of the map
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode("utf-8")

In [266]:
# create map of Madrid using latitude and longitude values
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_madrid['Latitude'], df_madrid['Longitude'], df_madrid['Neighbourhood']):
    label = folium.Popup(remove_accents(label), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    
map_madrid

#### Requesting Foursquare API for geospatial data <a class="anchor" id="requesting-foursquare"></a>

In [267]:
CLIENT_ID = 'BHJXSXUIL3QMVG5D40DTXTROBNKAMXXYNZSISUHVXAL3HXZO' # Foursquare ID
CLIENT_SECRET = 'FRLBLH40G0T4OWROEXCQU3MY35OZAIE3HXJ5J1X02RXNMTWP' # Foursquare Secret
ACCESS_TOKEN = 'N1LDGNZGUPHICQM1Y5LRR5RMQGULD1MLDZQIISPLTI0CL4I0' # FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('ACCESS_TOKEN:' + ACCESS_TOKEN)

Credentails:
CLIENT_ID: BHJXSXUIL3QMVG5D40DTXTROBNKAMXXYNZSISUHVXAL3HXZO
CLIENT_SECRET:FRLBLH40G0T4OWROEXCQU3MY35OZAIE3HXJ5J1X02RXNMTWP
ACCESS_TOKEN:N1LDGNZGUPHICQM1Y5LRR5RMQGULD1MLDZQIISPLTI0CL4I0


In [268]:
# Foursquare category IDs defined here: https://developer.foursquare.com/docs/build-with-foursquare/categories/
food_restaurants = '4d4b7105d754a06374d81259'
food_drink_stores = '4bf58dd8d48988d1f9941735'
food_categories = food_restaurants + ',' + food_drink_stores

In [269]:
##########################################################################################
# To Do:
# merge all types of restaurante-categories into 1 category named 'Restaurant',
# since for our analysis we are not interesting in the types of restaurant.
# Hint: if v['venue']['categories'][0]['name'].contains('Restaurant') then 'Restaurant'

In [322]:
# Function to merge all the restaurant-like categories into one category named 'Restaurant',
# and all the market-like shops into another category,
# since for our analysis we are not interested in the type of restaurant or market.
market_categories = ['Supermarket', 'Market', 'Farmers Market', 'Grocery Store', 'Big Box Store', 'Convenience Store', 'Food & Drink Shop']
def parseCategory(category):
    if 'Restaurant' in category or 'Burger' in category or 'BBQ' in category or 'Sandwich' in category or 'Pizza' in category:
        category = 'Restaurant'
    elif 'Bar' in category or 'Pub' in category or 'pub' in category or 'Joint' in category:
        category = 'Bar'
    elif 'Coffee' in category or 'Café' in category or 'Diner' in category:
        category = 'Coffee Shop'
    elif category in market_categories:
        category = 'Market'
    return category

# Function to find food-related venues in a given neighbourhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
#        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            food_categories, #'food', ################# ,
            radius, 
            LIMIT)
            
        # make the GET request
#        results = requests.get(url).json()["response"]['groups'][0]['items']
        results = requests.get(url).json()["response"]['venues']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            parseCategory(v['categories'][0]['name'])) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

__Run the above function to each neighbourhood__

In [323]:
madrid_venues = getNearbyVenues(names=df_madrid['Neighbourhood'],
                                   latitudes=df_madrid['Latitude'],
                                   longitudes=df_madrid['Longitude'],
                                   radius=radius)

print(madrid_venues.shape)
madrid_venues.head()

(649, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,San Isidro,40.39588,-3.73048,Mercadona,40.389170,-3.735899,Market
1,San Isidro,40.39588,-3.73048,La Cabra en el Tejado,40.409343,-3.709742,Bar
2,San Isidro,40.39588,-3.73048,El Imparcial,40.411929,-3.705782,Bistro
3,San Isidro,40.39588,-3.73048,MartinaCocina,40.410001,-3.707294,Coffee Shop
4,San Isidro,40.39588,-3.73048,Taberna El Buo,40.411333,-3.710443,Restaurant


In [324]:
madrid_venues[madrid_venues['Venue Category'] == 'Food & Drink Shop']

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


__Analysing each neighbourhood__

In [325]:
# one hot encoding
madrid_onehot = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
madrid_onehot['Neighbourhood'] = madrid_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [madrid_onehot.columns[-1]] + list(madrid_onehot.columns[:-1])
madrid_onehot = madrid_onehot[fixed_columns]

print(f'Size: {madrid_onehot.shape}')
madrid_onehot.head()

Size: (649, 23)


,Neighbourhood,Bakery,Bar,Beer Garden,Beer Store,Bistro,Breakfast Spot,Butcher,Chocolate Shop,Coffee Shop,...,Food,Gas Station,Gourmet Shop,Ice Cream Shop,Market,Pastry Shop,Restaurant,Snack Place,Steakhouse,Wine Shop
0,San Isidro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,San Isidro,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,San Isidro,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,San Isidro,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,San Isidro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


__Grouping rows by neighbourhood and finding frequency of each category__

In [326]:
madrid_grouped = madrid_onehot.groupby('Neighbourhood').mean().reset_index()
print(f'Size: {madrid_grouped.shape}')
madrid_grouped

Size: (13, 23)


,Neighbourhood,Bakery,Bar,Beer Garden,Beer Store,Bistro,Breakfast Spot,Butcher,Chocolate Shop,Coffee Shop,...,Food,Gas Station,Gourmet Shop,Ice Cream Shop,Market,Pastry Shop,Restaurant,Snack Place,Steakhouse,Wine Shop
0,Alameda de Osuna,0.060000,0.060000,0.00,0.00,0.00,0.000000,0.02,0.00,0.140000,...,0.020000,0.00,0.00,0.00,0.060000,0.00,0.620000,0.00,0.020000,0.00
1,Almagro,0.000000,0.080000,0.00,0.00,0.00,0.000000,0.00,0.02,0.240000,...,0.000000,0.00,0.04,0.00,0.260000,0.02,0.340000,0.00,0.000000,0.00
2,Arapiles,0.000000,0.040000,0.00,0.00,0.00,0.000000,0.00,0.02,0.240000,...,0.000000,0.00,0.04,0.00,0.200000,0.02,0.440000,0.00,0.000000,0.00
3,Cortes,0.040000,0.040000,0.00,0.02,0.00,0.000000,0.00,0.02,0.220000,...,0.000000,0.00,0.04,0.00,0.260000,0.02,0.340000,0.00,0.000000,0.00
4,Hellín,0.020000,0.040000,0.00,0.00,0.00,0.000000,0.00,0.00,0.160000,...,0.020000,0.00,0.00,0.00,0.160000,0.00,0.600000,0.00,0.000000,0.00
5,Hispanoamérica,0.040000,0.040000,0.00,0.00,0.00,0.000000,0.00,0.00,0.120000,...,0.000000,0.00,0.00,0.00,0.360000,0.00,0.380000,0.00,0.000000,0.00
6,Los Cármenes,0.020000,0.060000,0.02,0.00,0.00,0.020000,0.00,0.00,0.120000,...,0.000000,0.00,0.00,0.00,0.240000,0.00,0.520000,0.00,0.000000,0.00
7,Los Ángeles,0.060000,0.020000,0.00,0.00,0.00,0.040000,0.00,0.00,0.100000,...,0.000000,0.00,0.00,0.00,0.220000,0.00,0.540000,0.00,0.000000,0.00
8,Palomeras Bajas,0.020000,0.060000,0.00,0.00,0.00,0.020000,0.00,0.00,0.140000,...,0.000000,0.02,0.00,0.00,0.240000,0.00,0.480000,0.00,0.000000,0.00
9,Rosas,0.040816,0.040816,0.00,0.00,0.00,0.020408,0.00,0.00,0.081633,...,0.020408,0.00,0.00,0.00,0.122449,0.00,0.632653,0.00,0.020408,0.00


__Most common types of venue in each neighbourhood__

In [327]:
# Only show the 5 most common categories
num_top_venues = 5

for neighbourhood in madrid_grouped['Neighbourhood']:
    print("----"+neighbourhood+"----")
    temp = madrid_grouped[madrid_grouped['Neighbourhood'] == neighbourhood].T.reset_index()
    temp.columns = ['Venue','Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency': 2})
    print(temp.sort_values('Frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alameda de Osuna----
         Venue  Frequency
0   Restaurant       0.62
1  Coffee Shop       0.14
2       Bakery       0.06
3       Market       0.06
4          Bar       0.06


----Almagro----
          Venue  Frequency
0    Restaurant       0.34
1        Market       0.26
2   Coffee Shop       0.24
3           Bar       0.08
4  Gourmet Shop       0.04


----Arapiles----
          Venue  Frequency
0    Restaurant       0.44
1   Coffee Shop       0.24
2        Market       0.20
3  Gourmet Shop       0.04
4           Bar       0.04


----Cortes----
         Venue  Frequency
0   Restaurant       0.34
1       Market       0.26
2  Coffee Shop       0.22
3       Bakery       0.04
4          Bar       0.04


----Hellín----
         Venue  Frequency
0   Restaurant       0.60
1       Market       0.16
2  Coffee Shop       0.16
3          Bar       0.04
4       Bakery       0.02


----Hispanoamérica----
         Venue  Frequency
0   Restaurant       0.38
1       Market       0.36
2  Coffee

#### Loading census data file<a class="anchor" id="loading-census"></a>

### Neighbourhood segmentation <a class="anchor" id="neighbourhood-segmentation"></a>

### Census analysis <a class="anchor" id="census-analysis"></a>

## Results and discussion <a class="anchor" id="results-and-discussion"></a>

## Conclusion <a class="anchor" id="conclusion"></a>